In [95]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u

from photutils.aperture import SkyCircularAperture

import numpy as np
import os
import glob
import json 

import logging
logger = logging.getLogger("astroquery")
logger.setLevel(logging.CRITICAL)

home = '/home/jpcalderon/2024/CHANCES/'
datadir = home + 'data/'
tablesdir = home + 'tables/'
figuresdir = home + 'figures/'

bands = [ 'g', 'r', 'i', 'z' ]

In [153]:
param = {}
for file in glob.glob (datadir + 'NewLegacy/*fz'):
    param[os.path.basename(file)] = GetFWHM (file)

INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [158]:
with open('param.json', 'w') as convert_file: 
     convert_file.write(json.dumps(str(param)))

# reading the data from the file 
# with open('param.json') as f: 
#     d = f.read() 
# js = json.loads(d)

---
## Funciones

In [141]:
def GetFWHM (img):
    n, fwhm = 0, 1 * u.arcsec
    with fits.open ( img ) as hdul:
        data = hdul[1].data
        header = hdul[1].header

    coords = get_GaiaStars ( img, header )
    if len(coords) != 0:
        positions = SkyCoord( coords['ra'].tolist() * u.deg, coords['dec'].tolist() * u.deg, frame = 'fk5' )
    
        aperture = SkyCircularAperture ( positions, r = 1.0 * u.arcsec )
        aperstats = ApertureStats ( data, aperture, wcs = WCS(header) ) 
        n, fwhm = len(aperstats), aperstats.fwhm.mean()
    return n, fwhm.value, data.min(), data.max()

def get_GaiaStars ( img, header ):
    # Query tomado de: https://arxiv.org/pdf/1804.09378.pdf
    from astroquery.gaia import Gaia
    Gaia.ROW_LIMIT = 100

    ra, dec = str(header['CENTRA']), str(header['CENTDEC'])
    width = str(np.abs(header['CORN2RA']-header['CORN1RA']))
    height = str(np.abs(header['CORN3DEC']-header['CORN2DEC']))

    query = "SELECT * \
             from gaiaedr3.gaia_source \
             where 1 = CONTAINS( POINT ( 'ICRS', ra, dec ), BOX ( 'ICRS', " + ra + ", " + dec + ", " + width + ", " + height + " ) ) \
              AND parallax_over_error > 10 \
              AND phot_g_mean_flux_over_error>50 \
              AND phot_rp_mean_flux_over_error>20 \
              AND phot_bp_mean_flux_over_error>20 \
              AND phot_bp_rp_excess_factor < 1.3+0.06*power(phot_bp_mean_mag-phot_rp_mean_mag,2) \
              AND phot_bp_rp_excess_factor > 1.0+0.015*power(phot_bp_mean_mag-phot_rp_mean_mag,2) \
              AND visibility_periods_used>8 \
              AND astrometric_chi2_al/(astrometric_n_good_obs_al-5)<1.44*greatest(1,exp(-0.4*(phot_g_mean_mag-19.5)))"

    job = Gaia.launch_job_async ( query, verbose = False ).get_results()
    return job[['ra', 'dec']]